In [1]:
# import libraries
import sys
from sqlalchemy import create_engine
import sqlite3

import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
from datetime import date, timedelta
today = date.today()
prev_60 = (today - timedelta(600)).strftime("%Y-%m-%d")
print(prev_60, today)

2019-07-16 2021-03-07


In [3]:
def predict_data(stock):
  df = web.DataReader(stock+'.NS', data_source='yahoo', start=prev_60, end=today)
  data = df.filter(['Close'])
  dataset = data.values
  training_data_len = math.ceil(len(dataset)*.8)

  # Scaling the data
  scaler = MinMaxScaler(feature_range=(0,1))
  scaled_data = scaler.fit_transform(dataset)

  # Creating training dataset
  train_data = scaled_data[0:training_data_len, : ]
  x_train = []
  y_train = []
  for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

  x_train, y_train = np.array(x_train), np.array(y_train)
  x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

  # Building LSTM model
  model = Sequential()
  model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
  model.add(LSTM(50,return_sequences=False))
  model.add(Dense(25))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mean_squared_error')
  model.fit(x_train, y_train, batch_size=1, epochs=1, verbose=1)

  # Creating testing dataset
  test_data = scaled_data[training_data_len - 60: , :]
  x_test = []
  y_test = dataset[training_data_len:, :]
  for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
  x_test = np.array(x_test)
  x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1], 1))

  #Get the predictions
  predictions = model.predict(x_test)
  predictions = scaler.inverse_transform(predictions)

  train = data[: training_data_len]
  valid = data[training_data_len: ]
  valid['Predictions'] = predictions
  valid['Date'] = valid.index
  database_filename = stock+'.db'
  engine = create_engine('sqlite:///{}'.format(database_filename))

  table_name = database_filename.split('.')[0]
  valid.to_sql(table_name, engine, index=False, if_exists = 'replace')

In [4]:
stock_data = [
  "ADANIPORTS","ASIANPAINT","AXISBANK","BAJAJ-AUTO",
  "BAJFINANCE","BAJAJFINSV","BPCL","BHARTIARTL","BRITANNIA",
  "CIPLA","COALINDIA","DIVISLAB","DRREDDY","EICHERMOT",
  "GAIL","GRASIM","HCLTECH","HDFCBANK","HDFCLIFE","HEROMOTOCO",
  "HINDALCO","HINDUNILVR","HDFC","ICICIBANK","ITC","IOC","INDUSINDBK",
  "INFY","JSWSTEEL","KOTAKBANK","LT","M&M","MARUTI","NTPC","NESTLEIND",
  "ONGC","POWERGRID","RELIANCE","SBILIFE","SHREECEM","SBIN","SUNPHARMA","TCS",
  "TATAMOTORS","TATASTEEL","TECHM","TITAN","UPL","ULTRACEMCO","WIPRO"
]

In [5]:
for stock in stock_data:
  predict_data(stock)

265/265 [==============================] - 9s 23ms/step - loss: 0.0075


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


265/265 [==============================] - 9s 23ms/step - loss: 0.0057


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


265/265 [==============================] - 9s 24ms/step - loss: 0.0249


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


265/265 [==============================] - 10s 26ms/step - loss: 0.0079


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


265/265 [==============================] - 9s 26ms/step - loss: 0.0265


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
!zip -r data.zip .
from google.colab import files

files.download("data.zip")

  adding: .config/ (stored 0%)
  adding: .config/logs/ (stored 0%)
  adding: .config/logs/2021.03.01/ (stored 0%)
  adding: .config/logs/2021.03.01/14.34.56.151354.log (deflated 53%)
  adding: .config/logs/2021.03.01/14.35.29.900487.log (deflated 54%)
  adding: .config/logs/2021.03.01/14.35.09.556851.log (deflated 87%)
  adding: .config/logs/2021.03.01/14.34.37.071843.log (deflated 92%)
  adding: .config/logs/2021.03.01/14.35.14.752951.log (deflated 54%)
  adding: .config/logs/2021.03.01/14.35.29.380461.log (deflated 54%)
  adding: .config/.last_update_check.json (deflated 25%)
  adding: .config/config_sentinel (stored 0%)
  adding: .config/.last_survey_prompt.yaml (stored 0%)
  adding: .config/.last_opt_in_prompt.yaml (stored 0%)
  adding: .config/active_config (stored 0%)
  adding: .config/gce (stored 0%)
  adding: .config/configurations/ (stored 0%)
  adding: .config/configurations/config_default (deflated 15%)
  adding: HINDALCO.db (deflated 82%)
  adding: KOTAKBANK.db (deflated 82

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>